# 🚦 Traffic Manager AI - REST API
Using localhost.run (free SSH tunnel)

In [ ]:
# Cell 1: Install
!pip install ultralytics flask opencv-python-headless --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt
import os
if not os.path.exists('yolov5'):
    !git clone --depth 1 https://github.com/ultralytics/yolov5.git 2>/dev/null
for f,o,n in [('yolov5/utils/plots.py','import seaborn','#'),('yolov5/utils/plots.py','from scipy','#'),('yolov5/models/yolo.py','from utils.plots','#')]:
    try:
        with open(f,'r') as x: c=x.read()
        with open(f,'w') as x: x.write(c.replace(o,n))
    except: pass
# Generate SSH key for localhost.run
!mkdir -p ~/.ssh && ssh-keygen -t rsa -N '' -f ~/.ssh/id_rsa -q <<< y 2>/dev/null || true
print("✅")

In [ ]:
# Cell 2: Load Models
import torch, sys, os
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Device: {device}")

vehicle_model = tl_model = lp_detector = lp_ocr = None

try: vehicle_model = YOLO('yolo11m.pt').to(device); print("✅ Vehicle")
except Exception as e: print(f"⚠️ V: {e}")

try: tl_model = YOLO('/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/mhiot-dentinhieu-best-new.pt').to(device); print("✅ Traffic Light")
except Exception as e: print(f"⚠️ TL: {e}")

try:
    if 'yolov5' not in sys.path: sys.path.insert(0, os.path.abspath('yolov5'))
    lp_detector = torch.load('/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/LP_detector.pt', map_location=device, weights_only=False)['model'].float().eval()
    lp_ocr = torch.load('/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/LP_ocr.pt', map_location=device, weights_only=False)['model'].float().eval()
    if device=='cuda': lp_detector,lp_ocr = lp_detector.cuda(), lp_ocr.cuda()
    print("✅ LP")
except Exception as e: print(f"⚠️ LP: {e}")

print(f"\n📊 V:{'✅' if vehicle_model else '❌'} TL:{'✅' if tl_model else '❌'} LP:{'✅' if lp_detector else '❌'}")

In [ ]:
# Cell 3: Flask API
import cv2, numpy as np, time, threading
from flask import Flask, request, jsonify

app = Flask(__name__)
app.config['MAX_CONTENT_LENGTH'] = 50*1024*1024
CLASSES = ['car','truck','bus','motorcycle','bicycle']
trackers = {}

@app.route('/health')
def health(): return jsonify({'status':'ok','v':bool(vehicle_model),'tl':bool(tl_model),'lp':bool(lp_detector)})

@app.route('/detect', methods=['POST'])
def detect():
    try:
        if 'image' in request.files:
            img_bytes = request.files['image'].read()
            cam = request.form.get('camera_id','unknown')
            line_y = float(request.form.get('track_line_y',50))
        else:
            import base64; d=request.json
            img_bytes=base64.b64decode(d.get('image')); cam=d.get('camera_id','unknown'); line_y=d.get('track_line_y',50)
        frame = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_COLOR)
        if frame is None: return jsonify({'error':'bad'}),400
        h,w = frame.shape[:2]
        result = {'camera_id':cam,'dims':{'w':w,'h':h}}
        if cam not in trackers: trackers[cam]={'tr':{},'ct':{},'up':0,'down':0}
        t = trackers[cam]
        if vehicle_model:
            t0=time.time(); dets=[]; ly=int(h*line_y/100)
            for r in vehicle_model.track(frame,persist=True,verbose=False):
                for b in r.boxes:
                    c=vehicle_model.names[int(b.cls[0])]
                    if c not in CLASSES or float(b.conf[0])<0.5: continue
                    x1,y1,x2,y2=map(int,b.xyxy[0]); cy=(y1+y2)//2
                    det={'class':c,'conf':float(b.conf[0]),'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}}
                    if hasattr(b,'id') and b.id:
                        tid=int(b.id[0]); det['id']=tid
                        if tid in t['tr']:
                            py=t['tr'][tid]
                            d=1 if py<=ly<cy else (-1 if py>=ly>cy else 0)
                            if d and f"{tid}_{d}" not in t['ct']:
                                t['ct'][f"{tid}_{d}"]=1
                                if d==1: t['down']+=1
                                else: t['up']+=1
                        t['tr'][tid]=cy
                    dets.append(det)
            result['vehicle']={'dets':dets,'ms':(time.time()-t0)*1000,'up':t['up'],'down':t['down']}
        if tl_model:
            t0=time.time(); st=None; mx=0
            for r in tl_model(frame,verbose=False):
                for b in r.boxes:
                    if float(b.conf[0])>mx: mx=float(b.conf[0]); st=tl_model.names[int(b.cls[0])]
            result['traffic_light']={'status':st,'ms':(time.time()-t0)*1000}
        return jsonify(result)
    except Exception as e: return jsonify({'error':str(e)}),500

# Start Flask in thread
threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
print("✅ Flask on :5000")
time.sleep(2)

In [ ]:
# Cell 4: SSH Tunnel (localhost.run - FREE)
# This runs SSH tunnel in foreground - blocks and keeps alive
print("🚀 Starting localhost.run tunnel...")
print("═" * 60)
print("📋 Look for URL like: https://xxxxx.lhr.life")
print("═" * 60)
!ssh -o StrictHostKeyChecking=no -R 80:localhost:5000 localhost.run